# 1.載入並處理原始資料

In [1]:
import pandas as pd
from IPython.display import clear_output
import numpy as np

def getData() :
    df_text = pd.read_excel("CCQ1.xlsx")

    # 去空的文本
    empty_title = ((df_text["信用卡回饋描述（我們有整理過）"].isnull()) \
                   | (df_text["信用卡回饋描述（我們有整理過）"] == '') \
                   | (df_text["信用卡回饋描述（我們有整理過）"] == '0'))
    df_text = df_text[~empty_title]

    OA = df_text.loc[:, ["回饋類型原詞"]]
    QL = "回饋類型?"
    df_text = df_text.loc[:, ["信用卡回饋描述（我們有整理過）"]]

    for i in range(len(df_text)) : 
        #換行換｜　去空格
        df_text["信用卡回饋描述（我們有整理過）"][i] = df_text["信用卡回饋描述（我們有整理過）"][i].replace('\n', '｜').replace('\r', '')
    
    return df_text,QL,OA

# 2.半自動標記答案位置

In [2]:
from transformers import BertTokenizer

PRETRAINED_MODEL_NAME = "bert-base-multilingual-uncased"  # 指定繁簡中文 BERT-BASE 預訓練模型
# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

def MakeList(df,row) :
    L = []
    com = ""
    for i  in range(row, len(df_text)) :
        S = df["信用卡回饋描述（我們有整理過）"][i]
        sl = tokenizer.tokenize(S)
        oa = tokenizer.tokenize(OA["回饋類型原詞"][i])
        l = []
        while (1) :
            Q, s_p, e_p = find(sl,QL,i,oa)
            com = input("離開、跳過或繼續 ")
            if com == "exit" :
                break
            elif com == "next" :
                break
            elif com == "y" or com == "Y" :
                l.append(Q)
                l.append(S)
                l.append(s_p)
                l.append(e_p)
                S = "".join(sl[s_p:e_p])
                l.append(S)
                L.append(l)
                break
                
        if com == "exit" :
            return L
    return L
            
def find(sl,ql,i,oa) :
    clear_output()
    s_p = 0
    e_p = 0
    s = "".join(sl)
    a = "".join(oa)
    print(i)
    print(s)
    print(a)
    for j in range(len(sl)):
        if (sl[j]==oa[0]) :
            for k in range(len(oa)):
                if(oa[k]!=sl[j+k]) :
                    break
                elif(k==len(oa)-1) :
                    print(j,"",j+k+1)
                    s_p = j
                    e_p = j+k+1
                    break
                elif(j+k+1>=len(sl)):
                    break
        
    print('---------------------')
    Q = ql
    print(Q + "".join(sl[s_p:e_p]))    
    print('---------------------')
    while(1) :
        c = input("確認? ")
        if c == 'y' or c == 'Y' : #自動擷取的資料可以
            return Q, s_p, e_p
        elif c == 'h' or c == 'H' : #手動輸入
            while (1) :
                clear_output()
                print(i)
                print(s)
                print(a)
                for j in oa :
                    print(j," ",get_index(sl,j))
                s_p = input("輸入s_p：")
                e_p = input("輸入e_p：")
                if s_p.isdigit() and e_p.isdigit() :#防止有輸入不是數字
                    s_p = int(s_p)
                    e_p = int(e_p)
                else :
                    continue
                print('---------------------')
                print(Q + "".join(sl[s_p:e_p]))    
                print('---------------------')
                c = input("確認? ")
                if c == 'y' or c == 'Y' :
                    return Q, s_p, e_p

def get_index(lst=None, item=''): #取得
    tmp = []
    tag = 0
    for i in lst:
        if i == item:
            tmp.append(tag)
        tag += 1
    return tmp
        
df_text,QL,OA = getData()
L = MakeList(df_text,int(input("第幾筆? ")))

5
3.109/01/01~109/12/31##cos##tc##o加油站持聯名卡加油滿100元=3元多利金(0.3%)，每月上限100元
多利金
31  34
---------------------
回饋類型?多利金
---------------------
確認? y
離開、跳過或繼續 exit


In [4]:
print(L)

[['回饋類型?', '1.109/01/01~109/12/31蝦皮賣家新申辦蝦皮聯名卡，於蝦皮商城刷卡消費購買廣告儲值金，核卡後90天內皆可享10%現金回饋方案，每戶回饋上限100元。', 54, 58, '現金回饋'], ['回饋類型?', '2.109/01/01~109/12/31非蝦皮購物之一般消費單筆享0.5%蝦幣回饋。', 30, 32, '蝦幣'], ['回饋類型?', '3.109/01/01~109/12/31蝦皮站內消費，國泰世華/蝦皮商城提供最高4%蝦幣回饋，以下為級距說明:｜*國泰世華提供                               ｜4,999元以下=1%                            ｜5,000元~19,999元=1.5%                  ｜20,000元以上=2%｜*蝦皮商城提供｜非蝦皮商城=1%｜ 蝦皮商城=2%', 35, 37, '蝦幣'], ['回饋類型?', '1.109/01/01~109/12/31Costco賣場內消費滿100元=1元多利金(1%)', 27, 30, '多利金'], ['回饋類型?', '2.109/01/01~109/12/31Costco賣場外，一般消費滿200元=1元多利金(0.5%)', 30, 33, '多利金']]


# 3.將這次標記的檔案轉成 pandas

In [3]:
df_out = pd.DataFrame(L)
df_out.columns = ['Q','text','s_position', 'e_position', 'A'] # 欄位重新命名
df_out


,Q,text,s_position,e_position,A
0,回饋類型?,1.109/01/01~109/12/31蝦皮賣家新申辦蝦皮聯名卡，於蝦皮商城刷卡消費購買廣...,54,58,現金回饋
1,回饋類型?,2.109/01/01~109/12/31非蝦皮購物之一般消費單筆享0.5%蝦幣回饋。,30,32,蝦幣
2,回饋類型?,3.109/01/01~109/12/31蝦皮站內消費，國泰世華/蝦皮商城提供最高4%蝦幣回...,35,37,蝦幣
3,回饋類型?,1.109/01/01~109/12/31Costco賣場內消費滿100元=1元多利金(1%),27,30,多利金
4,回饋類型?,2.109/01/01~109/12/31Costco賣場外，一般消費滿200元=1元多利金...,30,33,多利金


# 4.將過去的 pandas 載入並添加這次的 pandas

In [4]:
df_out = pd.DataFrame(L)
df_out.columns = ['Q','text','s_position', 'e_position', 'A'] # 欄位重新命名
df_load = pd.read_csv("cardQ1.csv")
df_out = pd.concat([df_load,df_out])
df_out

,Q,text,s_position,e_position,A
0,回饋類型?,1.109/01/01~109/12/31蝦皮賣家新申辦蝦皮聯名卡，於蝦皮商城刷卡消費購買廣...,54,58,現金回饋
1,回饋類型?,2.109/01/01~109/12/31非蝦皮購物之一般消費單筆享0.5%蝦幣回饋。,30,32,蝦幣
2,回饋類型?,3.109/01/01~109/12/31蝦皮站內消費，國泰世華/蝦皮商城提供最高4%蝦幣回...,35,37,蝦幣
3,回饋類型?,1.109/01/01~109/12/31Costco賣場內消費滿100元=1元多利金(1%),27,30,多利金
4,回饋類型?,2.109/01/01~109/12/31Costco賣場外，一般消費滿200元=1元多利金...,30,33,多利金
...,...,...,...,...,...
6,回饋類型?,1.109/01/01~109/12/31哩程累積，長榮官網購票/機上免稅品/海外消費，啟動...,13,17,哩程累積
7,回饋類型?,2.109/01/01~109/12/31哩程累積，國內旅行社購買長榮機票/國外自由行/國外...,0,0,
8,回饋類型?,3.109/01/01~109/12/31哩程累積，一般消費，22元=1哩,0,0,
9,回饋類型?,1.109/01/01~109/12/31站外一般消費1%加油金回饋，享200元=1元加油金,21,24,加油金


# 5.儲存檔案

In [ ]:
df_out.to_csv("cardQ1.csv", index=False)  